13 avril 2021

LICIT (ENTPE/IFSTTAR)

PHD project: Observation, modeling and optimization of the operation of freeways merging.

Carlos GOMEZ PATINO.

This program allows the simulation to represente and observe the capacity  variation in freeways according to the variability of individuel parameteres by comparing different probability distributions.

In [2]:
import numpy as np
import math as m
import pandas as pd
import random
import matplotlib
import datetime
import scipy.stats
import collections
import scipy.interpolate as inter

matplotlib.use('TkAgg')

from matplotlib import pyplot as plt
from matplotlib import pyplot
from mpl_toolkits.mplot3d import Axes3D

from sklearn.linear_model import LinearRegression

In [24]:
# 1. INPUTS

######################################
# Simulation parameters: it is the general parametres of the simulation
    #%%! Total time of the simulation (in minutes)
T_sim  = 15  #[min] 

    #%%! Total distance to calculate (in meters)
x_sim = 12000 #[m]

#%%! Percentage insertion time of vehicles in the simulation [%time] 
ins_time_per = 0.75 
#%%! Percentage time of increasing flow rate  of the insertion time [%time] 
    # For a random slope 
inc_t_per_min = 0.7 # minimum value 0.6
inc_t_per_max = 0.7 # minimum value 0.7

######################################
    #%%! Calculation step of the simulation (in seconds < 0.1 [s])
t_step = 1 #[s] 
    #%%! Initial time of the simulation (in seconds)
t_0 = 0       #[s] 
    #%%! Initial position of the simulation (in meters)
x_0 = 0       #[m] 
    #%%! Replication quantity of a same scenario (same variability level)
replications = 100

######################################
# Roadway parameters and traffic flow variables
    # Fundamental diagram (FD) variables 
    #%%! Surcongestion waves speed (in meters/seconds)
w = 6        # [m/s]
    #%%! Maximal concentration or density of vehicles (=1/minimal distance) (in vehicles/meter)
k_max = 1/7.5  # [veh/m]
    #%%! Free flow speed (in meters/seconds)
v_ff  = 30     # [m/s]

    # traffic flow variables in the simulation
    #%%! Initial flow rate at t_0 (in vehicles/minutes)
q_0 = 22 #[veh/min]
        #%%! Roadway capacity of a simply lane (in vehicles/minutes)
Q_road = 40 #[veh/min]
  
######################################    
    # The proposed strategy in the study for generating the bottleneck is to
    # establish a speed limit zone on the roadway)
    
    #%%! Maximum input flow rate 
qm_max = 34 #[veh/min]

    #%%! Speed limit on the limitation zone (in meters/seconds)
v_lim = 10 # [m/s]
    #%%! Initial point of the limitation zone (in meters from the x_0)
x_lim = 4000 # [m]
    #%%! Random length of the limitation zone (in meters from the x_lim)
L_lim_min = 100 # [m] 
L_lim = 100 # [m]

    #%%! Localization of measure flow during the time, upstream and downstream of the limitation zone.
x_ups = 3850 # [m]
x_downs = 6200 # [m]
agg_time = 10 # [s]

    # Eddy's variables for the measure of flow
delta_x = 50 #[m]
    ### size of the Eddy's zone 
delta_t = 60 #[seconds]

#############################################
# Car-following model (cfm): 
# if Newell's model cfm= 'Ncf', if Tampere's model cfm= 'Tcf', if both models 'cfm=2cf'
cfm = 'Ncf'

# Individual parameters
    # Mean of each parameter used to define the normal distributions
    # associated with the random variables for small vehicles (sv) 
    # variability levels: N0(0%), NI(5%), NII(10%), NIII(15%)
    
    #%%! Mean and Standar deviation of desired stopping distance considered for sv (= 1/k_max from FD)(in meters)
avg_d = 1/k_max # [m]
std_d = 1.5 # std10 = 0.75, std20 = 1.5, std30 = 2.25

    #%%! Mean and Standar deviation of reaction time (from FD --> t= d/w) (in seconds)
avg_t = 1/(k_max*w)       # [s]
std_t = 0.25 # std10 = 0.125, std20 = 0.25, std30 = 0.375

    #%%! Mean and Standar deviation of desired maximal acceleration (in meters/seconds squared )
avg_a = 2.5 #2.5 maximum [m/s²]
std_a = 0.5 #std10 = 0.25, std20 = 0.5, std30 = 0.75

    # Tampere's parameters 
    #%%! Mean and Standar deviation of sensitivity to speed difference c1
avg_c1 = []; std_c1 = [] # max 0.05
    #%%! Mean and Standar deviation of sensitivity to desired distance c2 (between real distance and desired distance)
avg_c2 = []; std_c2 = [] # max 0.02
    #%%! Mean and Standar deviation of sensitivity to desired speed c3 (between actual speed and desired speed)
avg_c3 = []; std_c3 = [] # max 0.03

In [25]:
# 2. PROCESS

### Time stock
date_time()

# It consist of three steps: 1) synthetic data; 2) replications; and 3) variablity level 
# This means that different sets of synthetic data are generated based on the replications value. It's for each level of variability. 

# Random limited speed vector 
VL = np.random.uniform(v_lim,v_lim,replications)
[congestion_onset, flow_ups,flow_downs] = [[],[],[]]
[mean_dist,mean_react,mean_acce] = [[],[],[]]
[std_dist,std_react,std_acce] = [[],[],[]]

for vi_lim in VL:
         # Assignation of seed value for randomness
        seed = int(np.random.uniform(0,100000)) #seed
        np.random.seed(seed)

        Q_road = qm_max # maximum trafic demand 
        v_lim = vi_lim  # random limited speed 
        
        # Initial random inter-time between vehicles
        #%%! Flow rate profile increasing over time from q_0 to Q_road during half of the simulation time  
        [t_veh_inser, prof_flR, t_q_incr] = flow_profil(T_sim, q_0, Q_road,ins_time_per,inc_t_per_min,inc_t_per_max)

         #%%! Vehicles quantity on data set and initial headway time of each vehicle 
        [n_veh, t_inter_veh] = initial_headway_time(t_veh_inser, prof_flR)
        ####################################################################
        
        # Initial trajectory corresponding to the first vehicle (leader vehicle)
        # Assignment of random individual variables of the leader vehicle 
             #%%! 1. Variable of Minimum stopping distance (d_i), 2. Variable of desired speed in free flow (v_i)
             #%% 3. Variable of desired acceleration (a_i) 4. Variable of reaction time (t_reac_i)
        [cf_i,d_i,t_reac_i,w_i,a_i,c1_i,c2_i,c3_i] =  random_variables(avg_d, std_d, avg_t, std_t, avg_a, std_a,
                                                                     avg_c1,std_c1,avg_c2,std_c2, avg_c3,std_c3,cfm)
        [T_ldr_t_reac, T_ldr_t_step] = traj_leader(T_sim,x_sim,t_step,t_0,x_0, x_lim, L_lim, v_ff, v_lim, d_i,t_reac_i,a_i)
        [data_set_position, data_set_speed, data_set_acc] = traj_init(T_ldr_t_step, n_veh)
        ###################################################################
        
            # Generation of the trajectories data set of the follower vehicles according to the Newell's model
        #%%! Data set of the position and speed trajectories of the totality vehicles during the simulation time by calculation step \n",
        data_set = []
        data_set = data_set_trajectories(T_sim,x_sim,t_step,n_veh,t_0,t_inter_veh,x_0, v_ff, v_lim, x_lim, L_lim,cfm,T_ldr_t_reac,
                                                  avg_d, std_d, avg_t, std_t, avg_a, std_a,avg_c1,std_c1,avg_c2,std_c2, avg_c3,std_c3)
    
        ####################################################################
        if data_set[11] == 0:
            cong_t = output_var(agg_time, t_step, v_ff, delta_x, delta_t, data_set,L_lim, x_lim) 
            [veh_cong,cap_ups,cap_downs] =  capacities(data_set,cong_t,x_lim,v_ff)
            mean_d = np.mean(data_set[3]);  mean_t = np.mean(data_set[4]); mean_a = np.mean(data_set[5])
            st_dev_d = np.std(data_set[3]); st_dev_t = np.std(data_set[4]); st_dev_a = np.std(data_set[5])
            
            congestion_onset.append(cong_t); flow_ups.append(cap_ups); flow_downs.append(cap_downs)
            mean_dist.append(mean_d); mean_react.append(mean_t); mean_acce.append(mean_a)
            std_dist.append(st_dev_d); std_react.append(st_dev_t); std_acce.append(st_dev_a)
            
date_time()
data_results = pd.DataFrame(np.array([congestion_onset,flow_ups,flow_downs,mean_dist,mean_react,mean_acce,std_dist,std_react,std_acce]).T,columns = 
                           ['Cong_on','Cap_upstr','Cap_downstr', 'mean_dist','mean_react','mean_acce','std_dist','std_react','std_acce'])
data_results['variability_level'] = 2
data_results['parameter_var'] = 'all'
data_results['distribution'] = 'normal_ind'
data_results['limited_speed'] = v_lim

data_results.to_csv('data_std2_all_Normal_ind_v10.csv')

2021-07-06 08:18:08.862000


C:\Users\carlos.gomezpatino\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in less_equal
  del sys.path[0]
C:\Users\carlos.gomezpatino\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in less_equal
C:\Users\carlos.gomezpatino\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in less_equal
  


2021-07-06 11:38:38.832000


In [28]:
print np.mean(mean_dist)
print np.mean(mean_react)
print np.mean(mean_acce)
print np.mean(std_dist)
print np.mean(std_react)
print np.mean(std_acce)

7.509465753803487
1.2513643332027558
2.504599591030839
0.8658988352480433
0.1853471070799097
0.4868123786007959


In [ ]:
def cdf(flow_ups,flow_downs):
    
    n = len(flow_ups)
    p = np.array(range(n))/float(n)
    q_ups = np.sort(flow_ups)
    q_downs = np.sort(flow_downs)
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    plt.plot(q_ups, p, color= 'black')
    plt.plot(q_downs,p, color= 'gray')
    ax.set_ylabel('$F(x)$', fontsize = 12) 
    ax.set_xlabel('Flow [veh/min]', fontsize = 12)
    plt.grid(True)
#     plt.axis([28,33,-2,55])
    plt.legend(('Upstr capacity','Downstr capacity'))
    plt.title('CDF capacity: independent parameters with Normal fonction ',fontsize = 12)
#     fig.savefig('Fig.var_normal_inde.pdf', dpi=100)           
    return fig
    
cdf(flow_ups,flow_downs)
plt.show()

In [1]:
def cap_drop(flow_ups,flow_downs):
    
    [x_fit, y_fit,a,b] = linear_reg(flow_ups,flow_downs)
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    plt.plot(flow_ups, flow_downs, 'o', color= 'black')
    plt.plot(x_fit,y_fit,'-',color= 'r')
    plt.plot([18, 38],[18,38],'-',color= 'grey')
    ax.set_ylabel('$Cap downstr$', fontsize = 12) 
    ax.set_xlabel('$Cap upstr$', fontsize = 12)
    plt.grid(True)
    plt.axis([18,38,18,38])
#     plt.legend(('Upstr capacity','Downstr capacity'))
    plt.title('Capacity drop: independent parameters with Normal fonction ',fontsize = 12)
#     fig.savefig('Fig.var_normal_inde.pdf', dpi=100)           
    return fig

cap_drop(flow_ups,flow_downs)
plt.show()  

NameError: name 'flow_ups' is not defined

In [61]:
def linear_reg(x,y):
    data = pd.DataFrame(np.array([x,y]).T,columns = ['x','y'])
    data.sort_values(by=['x'],inplace=True)
    reg = scipy.stats.linregress(data['x'],data['y'])
    y_fit = [reg[0]*x+reg[1] for x in data['x']]
    return data['x'],y_fit,reg[0],reg[1]

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.hist(flow_ups, np.arange(28,33,0.2), color= 'black')
ax.hist(flow_downs, np.arange(28,33,0.2), color= 'gray')
ax.set_ylabel('All parameters variability', fontsize = 12) 
ax.set_xlabel('Flow [veh/min]', fontsize = 12)
plt.grid(True)
plt.axis([28,33,-2,55])
plt.legend(('Pre-breakdown capacity','Post-breakdown capacity'))
plt.title('Capacity distribution: independent parameters with Normal fonction ',fontsize = 12)
fig.savefig('Fig.var_normal_inde.pdf', dpi=100)
plt.show()

In [3]:
def veh0(data_set,cong_t):
    x1 = 3995
    v0 = 0
    for veh in (data_set[0].transpose()[1:-1]):
        tx = find_t(veh, x1)
        if tx > cong_t:
            break
        v0 += 1
        
    x2 = 4500
    vf = 0
    for veh in (data_set[0].transpose()[1:-1]):
        tx = find_t(veh, x2)
        if tx > 890:
            break
        vf += 1
    
        
    return v0, vf

In [4]:
def capacities(data_set,cong_t,x_lim,v_ff):
    [v0, vf] = veh0(data_set,cong_t)
    
    headways = np.diff(data_set[6])
    instant_debit = [60/tiv for tiv in headways]
    t = np.cumsum(headways)
    df = pd.DataFrame()
    df['q'] = instant_debit
    df['sma_q'] = df['q'].rolling(30, center=True, win_type='triang').mean()
    t_0 = cong_t - x_lim/v_ff

    for i in range(len(t)):
        if t_0 <= t[i]: 
            q_t = (t_0-t[i-1])*(df['sma_q'][i]-df['sma_q'][i-1])/(t[i]-t[i-1]) + df['sma_q'][i-1]
            break
            
    cap_ups = round(q_t,2)
    
    x = 4500
    times = []
    for veh in (data_set[0].transpose()[v0:vf]):
        tx = find_t(veh, x)
        times.append(tx)

    cap_downs = (len(times)-1)/(times[-1]- times[0])*60
    
    return v0,cap_ups,cap_downs

In [5]:
def output_var(agg_time, t_step, v_ff, delta_x, delta_t, data_set,L_lim, x_lim):
    
    [cong_onset] = [np.nan]
    ######################################################
     # Speed profil      
    X = np.arange(3750,3910,25) #meters
    v_w = v_ff*0.7 # 30% free speed reduction 
    t_w = []

    for xi in X:
        x_measure = xi
        [t, density, flow]  = var_macro_eddy(agg_time, t_step, v_ff, delta_x, delta_t, x_measure, data_set)
        speed = [round(qi/(ki*60),2) for qi,ki in zip(flow, density)]
#         ax.plot(t,speed)
        p = []
        for i in range(len(t)): 
            if speed[i] < v_w:
                p = t[i] - (t[i]-t[i-1])*float(v_w-speed[i])/(speed[i-1]-speed[i])
                t_w.append(p)
                break 

    # sucongestion waves speed 
    wi = [float(dxi)/dti for dxi, dti in zip(np.diff(X),abs(np.diff(t_w)))] 
    W = np.mean(wi)

    # congestion onset and measure times
    if t_w != []:
        cong_onset = (t_w[-1] - float(x_lim - X[-1])/W)*0.95

    return cong_onset 

In [6]:
# Fundamental diagram 
def fig_FD(namefig, flow_ups,density_ups,flow_downs,density_downs):
    ##########################################################################    
    fig1 = plt.figure()
    ax = fig1.add_subplot(111)
    ax.plot(density_ups,flow_ups,'o', color='red', alpha = 0.5) 
    ax.plot(density_downs,flow_downs,'o', color='green', alpha = 0.5)
    ax.plot([0,0.08],[np.nanmean(flow_ups), np.nanmean(flow_ups)], ':', color='red')
    ax.plot([0,0.08],[np.nanmean(flow_downs), np.nanmean(flow_downs)], ':', color='green')
    ax.plot([0,0.022, 0.136],[0,40,0], ':', color='black')
    
    CD = round(float(np.nanmean(flow_ups)-np.nanmean(flow_downs))/np.nanmean(flow_ups)*100,1)
    plt.text(0.08 , np.nanmean(flow_ups)-1, 'CD: '+ str(CD)+' %', fontsize=12) 
    plt.text(0.02 , 20, 'U: 30 m/s', fontsize=12) 
    plt.text(0.1, 18 , 'W: 6 m/s', fontsize=12)  
    plt.text(0.005, 41, 'Qmax: 40 ', fontsize=12) 
    plt.text(0.022, 2 , 'Kc: 0.022 ', fontsize=12)  
    plt.text(0.110, 2 , 'Kmax: 0.133', fontsize=12)  

    ax.set_ylabel('Flow [veh/min]', fontsize = 15)
    ax.set_xlabel('Density [veh/m]',fontsize = 15 ) 
    plt.title('FD '+ namefig ,fontsize = 15)
    plt.axis([0, 0.15 , 0, 50])
    
    return fig1

In [7]:
# Flow, speed and density curves upstream/downstream 

def fig_qvd(namefig, time_ups,flow_ups,density_ups, time_downs,flow_downs,density_downs):
    
    ##########################################################################
    fig1= plt.figure()
    [k_ups, t_ups, q_ups, v_ups] = [[],[],[],[]]

    t_ups = time_ups
    k_ups = density_ups
    q_ups = flow_ups
    v_ups = [(qi/(ki*60)) for qi,ki in zip(q_ups,k_ups)]
    ax1 = fig1.add_subplot(3,1,1)
    ax1.plot(t_ups,q_ups,'-')
    ax1.set_ylabel('Flow [veh/s]', fontsize = 8)
    ax1.axis([0, T_sim*60 , 0, 45])  
    plt.title('UPSTREAM POINT '+ namefig ,fontsize = 15)

    ax2 = fig1.add_subplot(3,1,2)
    ax2.plot(t_ups,v_ups,'-')
    ax2.set_ylabel('Speed [m/s]', fontsize = 8)
    ax2.axis([0, T_sim*60 , 0, 32])

    ax3 = fig1.add_subplot(3,1,3)
    ax3.plot(t_ups,k_ups,'-')
    ax3.set_ylabel('Density [veh/m]', fontsize = 8)
    ax3.set_xlabel('Time [s]',fontsize = 15 )
    ax3.axis([0, T_sim*60 , 0, 0.15])

    ##########################################################################
    fig2 = plt.figure()
    [k_downs, t_downs, q_downs, v_downs] = [[],[],[],[]]

    t_downs = time_downs
    k_downs = density_downs
    q_downs = flow_downs
    v_downs = [(qi/(ki*60)) for qi,ki in zip(q_downs,k_downs)]
    ax1 = fig2.add_subplot(3,1,1)
    ax1.plot(t_downs,q_downs,'-')
    ax1.set_ylabel('Flow [veh/s]', fontsize = 8)
    ax1.axis([0, T_sim*60 , 0, 45])  
    plt.title('DOWNSTREAM POINT ' + namefig ,fontsize = 15)

    ax2 = fig2.add_subplot(3,1,2)
    ax2.plot(t_downs,v_downs,'-')
    ax2.set_ylabel('Speed [m/s]', fontsize = 8)
    ax2.axis([0, T_sim*60 , 0, 32])

    ax3 = fig2.add_subplot(3,1,3)
    ax3.plot(t_downs,k_downs,'-')
    ax3.set_ylabel('Density [veh/m]', fontsize = 8)
    ax3.set_xlabel('Time [s]',fontsize = 15 )
    ax3.axis([0, T_sim*60 , 0, 0.15])

    return fig1, fig2

In [6]:
def points_DF(density_ups, flow_ups):
    kmax = max(density_ups)
    for i in range(len(density_ups)):
        if density_ups[i] == kmax:
            qmax  = flow_ups[i]
    return kmax,qmax 

In [7]:
# Registration date time 
def date_time():
    date = str(datetime.datetime.now())
    print date 
    return date

Input flow profil

In [8]:
# This function allows to define the incrase flow period and constante flow period of the profil
def flow_profil(T_sim, q_0, Q_road,ins_time_per,inc_t_per_min,inc_t_per_max):
#Flow rate profile increasing over time from q_0 to qi_max 
   
    #%%! Insertion time of vehicles (in seconds)
    t_veh_inser = int(ins_time_per*T_sim*60) # [sec] 
   #%%! Time of increasing flow rate during of the insertion time (in seconds)
    t_incr_flR = int(np.random.uniform(inc_t_per_min,inc_t_per_max)*t_veh_inser) # [sec] 
   #%%! Flow rate increment each second (in vehicles/second/second)
    delta_flR = float(Q_road-q_0)/(t_incr_flR) # [veh/s/s] 

  #%%! generation of the flow rate profile. Assignation of the flow rate values each second during insertion time.
    prof_flR = np.zeros((t_veh_inser,1)) #vecteur de la taille de temps simulation en secondes
    prof_flR[0]= q_0 # initial flow rate at t_0

    for i in range(1,len(prof_flR)):
        if i < int(t_incr_flR):
            prof_flR[i] += (prof_flR[i-1] + delta_flR) # Increasing flow rate during 60% of insertion time of vehicles)
        else:
            prof_flR[i] = Q_road # Constant flow rate during 40% of insertion time of vehicles)

    return t_veh_inser,prof_flR,t_incr_flR

In [9]:
# inter-vehicular time function: This function gives the inter_time of each vehicle according to the specified flow rate.
def initial_headway_time(t_veh_inser, prof_flR):
    n_veh = int(sum(prof_flR)/60) # quantity of vehicles 
    dates_entry = []
    veh_missing_gen = 0  # Vehicles number that should have been generated at the previous time step and they is kepted. 
    nP = 1  # First vehicle genered at time t_0
    
    for t in range(t_veh_inser):  
        veh_gen = int(prof_flR[t]+veh_missing_gen)

        # Generation of entry dates with an exponential distribution each second. 
        #The average time between two dates is the inverse value to quantity of vehicles generated. 
        dates = np.random.exponential(1/(prof_flR[t]+veh_missing_gen),veh_gen)
        #cumulation date each second 
        dates_cum = np.cumsum(dates)
        dates_cum_t = []
        [dates_cum_t.append(t+dates_cum[j]) for j in range(len(dates_cum))]
        [dates_entry.append(dates_cum_t[i]) for i in range(len(dates_cum_t))]
        dates=[]

        nP += veh_gen 
        veh_missing_gen = prof_flR[t]+(veh_missing_gen-veh_gen)

    dates_minute = [] # entry dates of vehicles each minute 
    [dates_minute.append(np.mean(dates_entry[i:i+179])) for i in range(0,len(dates_entry)-180,59)] # average of dates during two minute \n",
    t_headway = np.append(np.diff(dates_minute), 1)
    t_headway[-1] = t_headway[-2]

    return n_veh,t_headway

In [10]:
# Headway correction function: this function allows the correction of headway according to the reaction time (t_reac). The initial headway can't be less that the reaction time of the vehicle. \n",
# This is due to the desired minimum distance of the driver (d_i)
def entry_date(veh,t_init,t_inter_veh,t_reac_i,t_rem):     
    # Inter-vehicular  time  according to flow rate at t_i  
    headway_i = t_inter_veh[veh-1] - t_rem 

    #correction
    if headway_i < t_reac_i:
        t_rem = t_reac_i - headway_i
        headway_i = t_reac_i 
    else: 
        t_rem = 0  
        
    # departure time of the each follower vehicle 
    t_init += headway_i  
    
    return t_init

Random parameters

In [11]:
# Random parameters function
def random_variables(avg_d, std_d, avg_t, std_t, avg_a, std_a,
                     avg_c1,std_c1,avg_c2,std_c2, avg_c3,std_c3,cfm):
    
    [d_i,t_i,w_i,a_i,c1_i, c2_i,c3_i] = [[],[],[],[],[],[],[]]
    
    if cfm == '2cf':
        cf = np.random.poisson(0.5)
        if cf == 0:
            cfm = 'Ncf'
        else: 
            cfm = 'Tcf'        
            
    if cfm == 'Ncf':
        cf_i = 'Ncf'
        # Newell's parameters
        t_i_min = 0.7 #0.9, 0.7, 0.5
        t_i = np.random.normal(avg_t,std_t)
        while t_i <= t_i_min :
            t_i = np.random.normal(avg_t,std_t)
            
        d_i_min = 4  #5,4,3
        d_i = np.random.normal(avg_d,std_d)
        while d_i <= d_i_min :
            d_i = np.random.normal(avg_d,std_d)
        
        w_i  = d_i/t_i 
        
        a_i_min = 0.5 
        a_i = np.random.normal(avg_a,std_a)
        while a_i <= a_i_min :
            a_i = np.random.normal(avg_a,std_a)
        
    elif cfm == 'Tcf':
        cf_i = 'Tcf'
                # Tampere's parameters
        c1_i = np.random.normal(avg_c1,std_c1)
        c2_i = np.random.normal(avg_c2,std_c2)
        c3_i = np.random.normal(avg_c3,std_c3) 
        
        d_i  = np.random.normal(avg_d,std_d)
        t_i  = np.random.normal(avg_t,std_t)
        w_i  = d_i/t_i 
        a_i  = abs(np.random.normal(avg_a,std_a))

    return cf_i, d_i,t_i,w_i,a_i,c1_i,c2_i,c3_i

Newell's model 

In [23]:
# Newell's model 
def newell_car_fol(v_fwr_tt,x_fwr_t, x_lim, **kwargs ):
    
    t_reac = kwargs['t_reac_i']
    d = kwargs['d_i']
    ds = kwargs['ds']
    v = kwargs['v_i']
    a_max = kwargs['a_i']
    
    # relation between spacing and speed
    v_t = (ds-d)/t_reac
    
    if x_fwr_t <= x_lim:  
        if v_t < v:  # Car-following mode ( ds < dc) 
            return max(v_t,0)
        else:         # Free flow mode ( ds > dc)\n",
            return v#min(v,v_fwr_tt + a_max*(1-v_fwr_tt/v)*t_reac)
    else: 
        if v_t < v:  # Car-following mode ( ds < dc) 
#             return max(min(v_t,v_fwr_tt + a_max*(1-v_fwr_tt/v)*t_reac),0)
            return max(min(v_t,v_fwr_tt + a_max*t_reac),0)
        else:         # Free flow mode ( ds > dc)
            return min(v,v_fwr_tt + a_max*t_reac)

Tampere's model

Leader trajectory

In [12]:
# This fonction allows to create the leader's trajectory according the reaction time and the step time 
def traj_leader(T_sim, x_sim,t_step,t_0,x_0, x_lim, L_lim, v_ff, v_lim, d_i,t_reac_i,a_i):
    
            #%%! Variables initialisation of the leader's trajectory 
    [t_init,x_init] = [t_0,x_0] # Initial time and position
    
            #%%! Leader's trajectories ( Position, speed and acceleration trajectories during to the time each reaction time seconds))\n",
    T_ldr_t_reac = leader_trajectory(T_sim,x_sim,t_init, x_init, v_ff, v_lim, x_lim, L_lim, d=d_i, v=v_ff, t_reac=t_reac_i, a=a_i)
        #%%! Interpolation of the trajectories according the calculation step: Discretized trajectory for the representation in the next part 3\n",
    T_veh_t_reac = T_ldr_t_reac
    T_ldr_t_step = interp_trajectory(T_sim,x_sim,t_step, T_veh_t_reac) # Time, position and speed each calculatiion step time.\n",
    
    return T_ldr_t_reac, T_ldr_t_step

In [13]:
# This fonction allows to create the initial trajectory or leader trajectory 
def leader_trajectory(T_sim,x_sim,t_init, x_init,v_ff,v_lim,x_lim , L_lim, **kwargs):
    d = kwargs['d']
    v = kwargs['v']
    t_reac = kwargs['t_reac']
    a = kwargs['a']

    #%%! Periodes number of  t_reac on T_sim*60
    n_t_reac = int((T_sim*60-t_init)/t_reac)+1 ### --> int(round((T_sim*60-t_init)/t_reac))+1

    # Generation of the trajectoires chart  with size equal n_t_reac X 4 (trajectories quantity= time, position, speed and acceleration)\n",
    # chaque column is like: 1) T_ldr_t_reac; 2) x_ldr_t_reac; 3) v_ldr_t_reac; 4) a_ldr_t_reac
    T_ldr_t_reac = np.zeros((n_t_reac,4)) # Leader trajectories on the chart
    T_ldr_t_reac[:,:] = np.nan
    
    # Initialisation of the trayectories from x_0 and t_0
    [t, x_t,v_tt] = [t_init,x_init,v]

    for i in range(0,n_t_reac-1):   ### --> n_t_reac -1
        # Time profile each t_reac (T_ldr_t_reac)
        T_ldr_t_reac[i,0] = t #[t] 

        # Position assignation to the trajectory (x_ldr_t_reac)
        T_ldr_t_reac[i,1] = x_t #[m]

        # Identification of the limitation zone according to position variable 
        # to introduce speed values on speed trajectory  

        if  x_lim - (pow(v-v_lim,2)/4) < x_t < x_lim + L_lim:
            v_t = v_lim
            if (v_t-v_tt)/t_reac < -a:
                v_t = v_tt - a*t_reac  # individual speed limit 
        else:
            v_t = v  # individual desired speed 
            if (v_t-v_tt)/t_reac > a:
                    v_t = v_tt + a*t_reac  # individual speed limit 
            
         #%%! position increment during the reaction time  
        x_t += v_t * t_reac   

        # speed assignation to the trajectory (v_ldr_t_reac)
        T_ldr_t_reac[i,2] = v_t #[m/s]

        #%%! time increment according to the reaction time
        t += t_reac
        
    # Trajectory finalisation at simulation time (T_sim*60). This calcul is done due to that T_sim*60/t_reac can be not entire (offset)\n",
    t_rem = T_sim*60-t # Remaining time  to finish just at T_sim*60. It is true because t_rest is always a value positive.\n",
    T_ldr_t_reac[-1,0] = int(t+t_rem)  # t+t_rest should be exactly T_sim*60
    T_ldr_t_reac[-1,1] = x_t+ v_t * t_rem
    T_ldr_t_reac[-1,2] = v_t

    return T_ldr_t_reac

# T_ldr_t_reac = leader_trajectory(T_sim,x_sim,t_0,x_0,v_ff,v_lim,x_lim , L_lim, d = 7.1575, v = 30, t_reac = 1.56816) #t_reac = 7.157/6.587
# print T_ldr_t_reac

# plt.plot(T_ldr_t_reac[:,0],T_ldr_t_reac[:,1])
# plt.show()

Follower trajectory 

In [14]:
##### Follower vehicle  Function : This function allows to generate the follower's trajectory due to individual random parameters  \n",
def follower_trajectories(T_sim,x_sim, t_step, t_init,x_0, v_ff,v_lim,x_lim,L_lim_min,L_lim,T_ldr_t_reac, **kwargs):
    
    T_veh_t_reac = T_ldr_t_reac
    cf = kwargs['cf']
    t_reac = kwargs['t_reac']
    d = kwargs['d']
    a = kwargs['a']
    v = v_ff
    
    # Tampere's parameters
    c1_i = kwargs['c1']
    c2_i = kwargs['c2']
    c3_i = kwargs['c3']
    
    # random position and random longueur de la ZLV
    x_lim #= np.random.uniform(x_lim, x_lim + L_lim_min)
    L_lim #= np.random.uniform(L_lim - v_lim*t_reac, L_lim + v_lim*t_reac)
    broken = 0
    
    #%%! Periodes number of t_reac during the time period included from t_init to T_sim*60 
    n_t_reac = int((T_sim*60-t_init)/t_reac)+1

    # Generation of the trajectoires chart  with size equal n_t_reac X 4 (trajectories quantity= time, position, speed and acceleration)\n",
    # Chaque column is like: 1) t_fwr_t_reac; 2) x_fwr_t_reac; 3) v_fwr_t_reac; 4) a_fwr_t_reac
    T_fwr_t_reac = np.zeros((n_t_reac,4)) # Follower trajectories on the chart
    T_fwr_t_reac[:,:] = np.nan
    
    # Initialisation of the trayectories from x_0 and t_init
    [t,x_fwr_t, v_fwr_tt] = [t_init, x_0, v]

    for i in range(0,n_t_reac-1):          
         # Time profile each t_reac (t_fwr_t_reac)\n",
        T_fwr_t_reac[i,0] = t #

           # Position assignation to the trajectory of the follower (x_fwr_t_reac)\n",
        T_fwr_t_reac[i,1] = x_fwr_t #[m]\n",
        
        #%%! Extraction of values from the leader trajectory  according to t_reac of the follower vehicule from t_init. \
        v_veh_tt = v_fwr_tt
        [x_ldr_t, v_ldr_t, a_ldr_t] = find_x(t,t_reac, v_veh_tt,T_veh_t_reac)

        #%%! Security distance  between leader and follower at time t.\n",
        ds_t = x_ldr_t - x_fwr_t
        if ds_t <= d:
            print ds_t
            print d
            broken = 1 
            break
        
         #############################################
            # Model application
        if cf == 'Ncf': 
            # Speed prediction at t+t_reac according to the Newell's relation established between the security distance (at t and) reaction time 
            # on car-following model. 
            v_fwr_t = newell_car_fol(v_fwr_tt,x_fwr_t,x_lim, t_reac_i=t_reac, d_i=d, ds=ds_t, v_i=v, a_i=a)
        elif cf == 'Tcf':
            # Acceleration prediction at (t + t_reac) according to Tampere's model
            a_fwr_t = TMP_car_fol(v_fwr_tt,v_ldr_t, t_reac_i=t_reac, d_i=d, ds=ds_t, v_i=v, c1=c1_i, c2=c2_i, c3=c3_i)

            # Speed in the next step (t + t_reac)
            v_fwr_t = v_fwr_tt + a_fwr_t * t_reac #[m/s]
            
        #############################################
        
       # Identification of the limitation zone according to position variable 
        # to introduce speed values on speed trajectory  
        if  x_lim <= x_fwr_t  <= x_lim +  L_lim:
            if v_fwr_t > v_lim:
                v_fwr_t = v_lim   # individual speed limit  
        
            # Speed assignation to the trajectory of the follower (v_fwr_t_reac)
        T_fwr_t_reac[i,2] = v_fwr_t #[m/s]
        v_fwr_tt = v_fwr_t
        #%%! time increment according to the reaction time
        t += t_reac #[t]
        #%%! Position increment during the reaction time  
        x_fwr_t += v_fwr_tt * t_reac #[m]

        # Trajectories finalisation at simulation time (T_sim*60). This calcul is done due to that T_sim*60/t_reac can be not entire (offset)\n",
    t_rem = T_sim*60-t # Remaining time  to finish just at T_sim*60. It is true because t_rem is always a value positive.\n",
    T_fwr_t_reac[-1,0] = t+t_rem  # t+t_rest should be exactly T_sim*60
    T_fwr_t_reac[-1,1] = x_fwr_t+v_fwr_tt * t_rem
    T_fwr_t_reac[-1,2] = v_fwr_tt

    return T_fwr_t_reac, broken

# T_fwr_t_reac = follower_trajectories(T_sim,x_sim,t_step, 1.45 , 0, v_ff,v_lim,x_lim , L_lim_min, L_lim,T_ldr_t_reac, d = 8.094464568, v = 30, t_reac = 1.7, a = 2.54)                            
# # print T_fwr_t_reac[0:20]
# plt.plot(T_fwr_t_reac[:,0],T_fwr_t_reac[:,1])
# plt.show()

In [22]:
# Interpolation function: This function allows to interpolate the trajectories according to calculation step of the simulation (t_step)\n",

def interp_trajectory(T_sim, x_sim, t_step, T_veh_t_reac):
     
    t_init = T_veh_t_reac[0,0]
    x_init = T_veh_t_reac[0,1]
    v = T_veh_t_reac[0,2]
    
    # t_init according to t_step (Periodes number of t_step during t_init )
    t_init_step = int(t_init/t_step)+1
    
    #%%! Periodes number of t_step during all time T_sim*60 
    n_t_step = int((T_sim*60)/t_step)+1 
    
    # Generation of the trajectoires chart  with size equal n_t_step X 4 (trajectories quantity= time, position, speed and acceleration)
    # Chaque column is like: 1) t_veh_t_step; 2) x_veh_t_step; 3) v_veh_t_step; 4) a_veh_t_step
    T_veh_t_step = np.zeros((n_t_step,4)) # vehicle trajectories on the chart
    T_veh_t_step[:,:] = np.nan 
    
    # Trajectory initialisation from x_0 and t_init
    t = t_init_step * t_step 
    v_veh_tt = v
    t_reac = T_veh_t_reac[3,0] - T_veh_t_reac[2,0]

    for i in range(t_init_step,n_t_step-1):  
        # Time profile each t_step 
        T_veh_t_step[i,0] = t #[t]
        #%%! Extraction of values from the  trajectory t_reac according to t_step of a vehicule (from t_0). 
        # It will be due to the interporlation between two values of the vehicle trajectory each t_step. 
        [x_veh_t,v_veh_t, a_veh_t] = find_x(t,t_reac, v_veh_tt, T_veh_t_reac)
        # Position assignation to the trajectory of the vehicle
        T_veh_t_step[i,1] = x_veh_t #[m]

        # Speed assignation to the trajectory of the vehicle
        T_veh_t_step[i,2] = v_veh_t #[m/s]
        v_veh_tt = v_veh_t
        
        # acceleration assignation to the trajectory of the vehicle
        T_veh_t_step[i,3] = a_veh_t #[m/s^2]

        #%%! time increment according to the calculation step 
        t += t_step
        
        if x_veh_t > x_sim: #maximum distance calculation (time calculation reduction)
            break
            
    t_rem = T_sim*60-(t-t_step) # Remaining time  to finish just at T_sim*60. It is true because t_rem is always a value positive.\n",
    T_veh_t_step[-1,0] = (t-t_step) + t_rem # t+t_rest should be exactly T_sim*60\n",
    T_veh_t_step[-1,1] = x_veh_t + v_veh_t * t_rem
    T_veh_t_step[-1,2] = v_veh_t
    
    return T_veh_t_step

# T_veh_t_step_L = interp_trajectory(T_sim,x_sim, t_step, T_fwr_t_reac)
# print T_veh_t_step_L


In [21]:
# This function allows to calculate x  according t for the interpolation between two calcul points 
def find_x(t,t_reac, v_veh_tt, T_veh_t_reac):
# This values are named above and below of the t
    for k in range(len(T_veh_t_reac)): 
        if T_veh_t_reac[k,0] > t: 
            v = (T_veh_t_reac[k,1]-T_veh_t_reac[k-1,1])/(T_veh_t_reac[k,0]-T_veh_t_reac[k-1,0])
            x = T_veh_t_reac[k-1,1] + v*(t-T_veh_t_reac[k-1,0])
            a = round((v-v_veh_tt)/t_reac,4)
            break  
            
    return x,v,a

Vehicle trajectories data set

In [15]:
# This fonction allows to initialize the data set for the position and speed trajectories
def traj_init(T_ldr_t_step, n_veh):
    
    #%%! Data set initialisation of the position trajectories: matrix contains the distance trajectory of all vehicles during the simulation time..\n",
    data_set_position      = np.zeros((len(T_ldr_t_step), n_veh+1)) # Time and position of each vehicle\n",
    data_set_position[:,0] = T_ldr_t_step[:,0] # The first column contains the time each calculation step \n",
    data_set_position[:,1] = T_ldr_t_step[:,1] # Assignation of the position trajectory of the first vehicle (leader vehicle)\n",
    
    #%%! Data set initialisation of the speed trajectories: matrix contains the speed trajectory of all vehicles during the simulation time.\n",
    data_set_speed      = np.zeros((len(T_ldr_t_step), n_veh+1)) # Time and speed of each vehicle\n",
    data_set_speed[:,0] = T_ldr_t_step[:,0] # The first column contains the time each calculation step\n",
    data_set_speed[:,1] = T_ldr_t_step[:,2] # Assignation of the speed trajectory of the first vehicle (leader vehicle)\n",
    
    #%%! Data set initialisation of the speed trajectories: matrix contains the speed trajectory of all vehicles during the simulation time.\n",
    data_set_acc = []
    data_set_acc      = np.zeros((len(T_ldr_t_step), n_veh+1)) # Time and acceleration of each vehicle\n",
    data_set_acc[:,0] = T_ldr_t_step[:,0] # The first column contains the time each calculation step\n",
    data_set_acc[:,1] = T_ldr_t_step[:,3] # Assignation of the acceleration trajectory of the first vehicle (leader vehicle)\n",
    
    return data_set_position, data_set_speed, data_set_acc

In [16]:
def data_set_trajectories(T_sim,x_sim,t_step,n_veh,t_0,t_inter_veh,x_0, v_ff, v_lim, x_lim, L_lim,cfm, T_ldr_t_reac,
                          avg_d, std_d, avg_t, std_t, avg_a, std_a,avg_c1,std_c1,avg_c2,std_c2, avg_c3,std_c3):
    
    [cf,distance,reac_time,acceleration,c1,c2,c3,headway] = [[],[],[],[],[],[],[],[0]]

    t_rem  = 0 # Variable of remaining time if headway is less that reaction time (used later for headway correction) \n",
    t_init = t_0
    
    for veh in range(1,n_veh): 
        # Individual random variables of each vehicle
        [cf_i,d_i,t_reac_i,w_i,a_i,c1_i,c2_i,c3_i] =  random_variables(avg_d, std_d, avg_t, std_t, avg_a, std_a,
                                                                     avg_c1,std_c1,avg_c2,std_c2, avg_c3,std_c3,cfm)
        
        cf.append(cf_i); distance.append(d_i); reac_time.append(t_reac_i); acceleration.append(a_i)
        c1.append(c1_i); c2.append(c2_i); c3.append(c3_i)
        
        # Initial time of the departure of the each vehicle 
        t_init = entry_date(veh,t_init,t_inter_veh,t_reac_i,t_rem)
        headway.append(t_init)

        # Follower's trajectories according to car-following model\n",
        [T_fwr_t_reac, broken] = follower_trajectories(T_sim,x_sim, t_step, t_init,x_0, v_ff,v_lim,x_lim ,L_lim_min,L_lim, T_ldr_t_reac, 
                                             cf = cf_i, t_reac=t_reac_i,d=d_i,a=a_i,c1=c1_i,c2=c2_i,c3=c3_i)
                    
        # Interpolation of the trajectories according the calculation step 
        T_veh_t_reac = T_fwr_t_reac 
        T_fwr_t_step = interp_trajectory(T_sim,x_sim,t_step, T_veh_t_reac)
        # Assignation of the position trajectory of the vehicle i to the data set of the speed trajectories.\n",
        data_set_position[:,(veh+1)] = T_fwr_t_step[:,1]
        # Assignation of the speed trajectory of the vehicle i to the data set of the speed trajectories. \n",
        data_set_speed[:,(veh+1)] = T_fwr_t_step[:,2] 
        # Assignation of the acceleration trajectory of the vehicle i
        data_set_acc[:,(veh+1)] = T_fwr_t_step[:,3]
        
        if broken == 1:
            break
        
        T_ldr_t_reac = T_fwr_t_reac

    return data_set_position,data_set_speed,data_set_acc, distance, reac_time,acceleration, headway,c1,c2,c3,cf,broken

In [20]:
# This function allows to identify the time at x_i
def find_t(veh, xi):
    x_measure = xi
    # interpolation 
    for t in range(len(veh)):
        if x_measure <= veh[t]:
            t_x = ((x_measure - veh[t-1])*(t-(t-1))/(veh[t]-veh[t-1])) + (t-1)  # time 
            break  
    return t_x

Flow and density profils

In [17]:
# # This function allows to identify the time at x_measure
def f_t(t_step,t11,t22,x11,x22,veh):
    [ta, tb] = [[],[]]
    # lower distance of the eddie's zone
    for t in range(len(veh)): 
        if x11 <= veh[t]:
            ta = ((x11 - veh[t-1])*(t-(t-1))/(veh[t]-veh[t-1])) + (t-1)  # time 
            break
    i = t       
    #  higher distance of the eddie's zone 
    for t in range(i,len(veh)): 
        if x22 <= veh[t]:
            tb = ((x22 - veh[t-1])*(t-(t-1))/(veh[t]-veh[t-1])) + (t-1)  # time 
            break
            
    return ta, tb

In [18]:
# This function allows to calculate x  according t_x
def f_x(veh,**kwargs):
    ta = kwargs['t_a']
    tb = kwargs['t_b']
# This values are named above and below of the t
    for k in range(len(veh)): 
        if k >= ta:
            v = float(veh[k]-veh[k-1])/(k-(k-1))
            xa = veh[k-1] + v*(ta-k)
            break  
            
    for j in range(k,len(veh)):   
        if j >= tb:
            v = float(veh[j]-veh[j-1])/(j-(j-1))
            xb = veh[j-1] + v*(tb-j)
            break  
            
    return xa,xb,ta,tb

In [19]:
def var_macro_eddy(agg_time, t_step, v_ff, delta_x, delta_t, x_measure, data_set):
    # We calcule the average of the flow rate and speed for the next 60 seconds after the first vehicule arrive to x_mesure downstream the LZ. 
    #%%! Variables downstream of the speed limitation zone

    # Position trajectories of the vehicles platoon each second
    data_set_x = data_set[0]

    # Eddie's square contruction of the measure
    [t0,tf] = [[],[]]
     #%%! first vehicle identification
    veh0 = data_set_x.transpose()[1]
    for t in range(len(veh0)):
        if x_measure  <= veh0[t]: # starting point of the measure
            t0 = t - 1      # time cordinate [seconds*t_step]  
            x = x_measure # space cordinate [m]
            break
            
        #%%! last vehicle identification
    vehf = data_set_x.transpose()[-1]
    for t in range(len(vehf)):
        if x_measure  <= vehf[t]: # starting point of the measure
            tf = t - 1      # time cordinate [seconds*t_step]  
            x = x_measure # space cordinate [m]
            break     
        elif tf == []: 
            tf = len(vehf)-1
            
            
    [time_x_measure, flow_x_measure, density_x_measure] = [[],[],[]]
    for time in range(t0, int(tf-float(delta_x/v_ff)/t_step-float(delta_t/t_step)),int(float(agg_time/t_step))): # time periods of agg_time seconds
        
        # Dynamical Eddie's square
        t11 = time # time cordinate [seconds*t_step] 
        x11 = x  # space cordinate [m]
        
                 #%%! second corner of the Eddie's square 
        t12 = int(t11 + float(delta_x/v_ff)/t_step) # time cordinate [seconds*t_step] 
        x12 = x11 + delta_x # space cordinate [m]

            #%%! third corner of the Eddie's square
        t21 = int(t11 + float(delta_t/t_step))  # time cordinate [seconds*t_step] 
        x21 = x11  # space cordinate [m]

            #%%! fourth corner of the Eddie's square
        t22 = int(t21 + float(delta_x/v_ff)/t_step) # time cordinate [seconds*t_step] 
        x22 = x21 + delta_x # space cordinate [m]

            # space traveled for each vehicle inside the Eddie's square
        [t_veh, l_veh] = [[],[]] # individual travel time and travel distance 
        for veh in (data_set_x.transpose()[1:-1]):     
                  # initialisation variables
            [ta,tb,ta2,tb2,xa,xb] = [0,0,0,0,0,0]
            [ta,tb] = f_t(t_step,t11,t22,x11,x22,veh)
            
            # Last vehicle which arrives to x_measure before t21
            if ta > t21: 
                break
                
             # interpolation: trajectories inside of Eddie's zone
            if ta<t11 and t12<tb<=t22:
                ti = (delta_x*(t11-ta))/(delta_t*v_ff-delta_x)+t11
                [xa,xb,ta2,tb2] = f_x(veh,t_a=ti,t_b=tb)

            elif t11<=ta<t21 and t12<tb<=t22:
                [xa,xb,ta2,tb2] = f_x(veh,t_a=ta,t_b=tb)

            elif t11<=ta<t21 and t22<tb:
                ti = (delta_x*(t21-ta))/(delta_t*v_ff-delta_x)+t22
                [xa,xb,ta2,tb2] = f_x(veh,t_a=ta,t_b=ti)
                
            t_veh.append(tb2*t_step-ta2*t_step)
            l_veh.append(xb-xa)

        # average density calculation 
        k_x_measure = float(sum(t_veh))/(delta_x*delta_t) #[veh/m]
        # average flow calculation 
        q_x_measure = 60*float(sum(l_veh))/(delta_x*delta_t) #[veh/min]
        
        time_x_measure.append(time*t_step+int(agg_time/2))    
        density_x_measure.append(k_x_measure)
        flow_x_measure.append(q_x_measure)

    return time_x_measure,density_x_measure, flow_x_measure 